In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import settings as conf

In [3]:
output_dir = os.path.join(conf.DELIVERABLES_DIR, 'roc_validation', 'classifier_tables')
os.makedirs(output_dir, exist_ok=True)

# Load gene mappings

In [5]:
with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load fastENLOC results

In [6]:
spredixcan_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, 'fastenloc-torus-rcp.pkl.xz')
display(spredixcan_genes_associations_filename)

spredixcan_genes_associations = pd.read_pickle(spredixcan_genes_associations_filename)

'/mnt/phenomexcan_base/gene_assoc/fastenloc-torus-rcp.pkl.xz'

In [ ]:
#spredixcan_genes_associations = spredixcan_genes_associations.rename(index=genes_mapping_1)

In [7]:
spredixcan_genes_associations.shape

(37967, 4091)

In [8]:
spredixcan_genes_associations.head(5)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,2.173200e-07,0.000036,1.020227e-10,0.000020,0.001497,3.426770e-11,1.312300e-10,1.508600e-09,1.114500e-05,4.164600e-09,...,0.001,NaN,0.000,0.0,NaN,0.001,NaN,NaN,NaN,0.0
ENSG00000000457,1.343900e-06,0.000692,8.216060e-11,0.000089,0.004363,1.342669e-10,4.223476e-10,2.036232e-08,4.598924e-04,3.570850e-08,...,0.000,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN
ENSG00000000460,8.757750e-06,0.001713,1.119010e-10,0.000016,0.003566,1.533228e-10,3.712300e-10,6.505700e-09,1.830200e-05,6.002900e-08,...,0.000,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN
ENSG00000000938,7.197000e-08,0.000446,3.573442e-10,0.000006,0.004429,2.588022e-08,3.461500e-10,9.110200e-09,1.351400e-05,9.878000e-09,...,0.263,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,0.0
ENSG00000000971,2.720000e-07,0.000310,1.397300e-10,0.000182,0.003160,3.940395e-11,1.785600e-10,2.329954e-10,5.325000e-07,4.958740e-08,...,0.000,0.0,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.0


# Load OMIM silver standard

In [16]:
omim_silver_standard = pd.read_csv(os.path.join(conf.DATA_DIR, 'omim_silver_standard.tsv'), sep='\t')

In [17]:
omim_silver_standard = omim_silver_standard.dropna(subset=['ensembl_gene_id', 'trait', 'pheno_mim'])

In [18]:
display(omim_silver_standard.shape)
display(omim_silver_standard.head())

(7809, 7)

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


### Save list of mapped UK Biobank traits

In [19]:
from entity import Trait

In [20]:
ukb_traits_mapped = pd.Series(omim_silver_standard['trait'].unique())
display(ukb_traits_mapped.head())

0               M41-Diagnoses_main_ICD10_M41_Scoliosis
1            H80-Diagnoses_main_ICD10_H80_Otosclerosis
2    20002_1226-Noncancer_illness_code_selfreported...
3    I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic...
4    20002_1136-Noncancer_illness_code_selfreported...
dtype: object

In [21]:
ukb_traits_mapped.shape

(107,)

In [22]:
ukb_codes_mapped = [Trait(full_code=t).code for t in ukb_traits_mapped]

In [23]:
len(ukb_codes_mapped)

107

In [24]:
ukb_codes_mapped[:5]

['M41', 'H80', '20002_1226', 'I25', '20002_1136']

In [25]:
# This is needed to run gwas2gene (Yanyu's scripts)
pd.Series(ukb_codes_mapped).to_csv('/mnt/tmp/selected_ukb_traits_omim.txt', index=False, header=False)

# Read gwas2gene results

In [26]:
from glob import glob

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [28]:
readRDS = robjects.r['readRDS']

In [29]:
f_files = glob(os.path.join(conf.OMIM_SILVER_STANDARD_GWAS_TO_GENE_DIR, '*.rds'))
display(len(f_files))

if len(f_files) != len(ukb_codes_mapped):
    print(f'WARNING: some files are not there. {len(ukb_codes_mapped)} expected, {len(f_files)} found.')

99

In [30]:
gwas2genes_results = {}

for f in f_files:
    f_base = os.path.basename(f)
    f_code = f_base.split('.')[0]
    
    #print(f_base)
    rds_contents = readRDS(f)
    
    if len(rds_contents[1]) > 0:
        f_gene_list = list(rds_contents[1][0].iter_labels())
    else:
        print(f'{f_code}: empty')
        f_gene_list = []
    
    gwas2genes_results[f_code] = spredixcan_genes_associations.index.intersection(set(f_gene_list))

In [31]:
len(gwas2genes_results)

99

In [32]:
gwas2gene_all_genes = []

for k in gwas2genes_results.keys():
    gwas2gene_all_genes.extend(gwas2genes_results[k])

display(len(gwas2gene_all_genes))

gwas2gene_all_genes = set(gwas2gene_all_genes)
display(len(gwas2gene_all_genes))

# gwas2gene_all_genes = spredixcan_genes_associations.index.intersection(gwas2gene_all_genes)
# display(len(gwas2gene_all_genes))

19709

9722

# Create list of UKB-OMIM traits

In [33]:
omim_silver_standard.head()

,trait,pheno_mim,mim,entry_type,entrez_gene_id,approved_gene_symbol,ensembl_gene_id
0,M41-Diagnoses_main_ICD10_M41_Scoliosis,101800,188830,gene,5573,PRKAR1A,ENSG00000108946
1,M41-Diagnoses_main_ICD10_M41_Scoliosis,102500,600275,gene,4853,NOTCH2,ENSG00000134250
2,M41-Diagnoses_main_ICD10_M41_Scoliosis,105830,601623,gene,7337,UBE3A,ENSG00000114062
3,M41-Diagnoses_main_ICD10_M41_Scoliosis,108120,190990,gene,7169,TPM2,ENSG00000198467
4,M41-Diagnoses_main_ICD10_M41_Scoliosis,108145,613629,gene,63895,PIEZO2,ENSG00000154864


# Create PrediXcan classifier table

In [34]:
_tmp = omim_silver_standard[['trait', 'ensembl_gene_id']]
ukb_traits_common = _tmp['trait'].unique()

omim_true_classes = _tmp[['trait', 'ensembl_gene_id']].drop_duplicates()
omim_true_classes = omim_true_classes.assign(omim_value=1)
omim_true_classes = omim_true_classes.set_index(['trait', 'ensembl_gene_id'])

In [35]:
len(ukb_traits_common)

107

In [36]:
omim_true_classes.shape

(7046, 1)

In [37]:
omim_true_classes.head()

omim_value
trait                                  ensembl_gene_id            
M41-Diagnoses_main_ICD10_M41_Scoliosis ENSG00000108946           1
                                       ENSG00000134250           1
                                       ENSG00000114062           1
                                       ENSG00000198467           1
                                       ENSG00000154864           1

In [38]:
len(ukb_traits_common)

107

In [39]:
index_tuples = []

for t in ukb_traits_common:
    t_code = Trait(full_code=t).code
    if t_code not in gwas2genes_results:
        continue
    
    for g in gwas2genes_results[t_code]:
        index_tuples.append((t, g))

In [40]:
len(index_tuples)

19709

In [41]:
index_tuples[:5]

[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000012504'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075089'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000075336'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000078401'),
 ('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263')]

In [42]:
classifier_index = pd.MultiIndex.from_tuples(
    index_tuples,
    names=['ukb_efo', 'gene']
)

In [43]:
len(gwas2gene_all_genes)

9722

In [44]:
classifier_index.shape

(19709,)

In [45]:
predixcan_classifier_df = pd.DataFrame(index=classifier_index, columns=['score', 'predicted_class', 'true_class'])

In [46]:
predixcan_classifier_df = predixcan_classifier_df.sort_index()

In [47]:
predixcan_classifier_df.shape

(19709, 3)

In [48]:
predixcan_classifier_df['true_class'] = 0

In [49]:
predixcan_classifier_df.head()

score predicted_class  true_class
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617   NaN             NaN           0
                            ENSG00000003756   NaN             NaN           0
                            ENSG00000004534   NaN             NaN           0
                            ENSG00000004838   NaN             NaN           0
                            ENSG00000004897   NaN             NaN           0

In [50]:
true_classes = omim_true_classes.squeeze()
display(true_classes.shape)
display(true_classes.head())

(7046,)

trait                                   ensembl_gene_id
M41-Diagnoses_main_ICD10_M41_Scoliosis  ENSG00000108946    1
                                        ENSG00000134250    1
                                        ENSG00000114062    1
                                        ENSG00000198467    1
                                        ENSG00000154864    1
Name: omim_value, dtype: int64

In [51]:
predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(true_classes.index), 'true_class'] = 1

In [52]:
assert predixcan_classifier_df['true_class'].isna().sum() == 0

In [53]:
# some testing
predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis',)].head()

,score,predicted_class,true_class
gene,,,
ENSG00000012504,NaN,NaN,0
ENSG00000075089,NaN,NaN,0
ENSG00000075336,NaN,NaN,0
ENSG00000078401,NaN,NaN,0
ENSG00000090263,NaN,NaN,0


In [54]:
true_classes.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234')]

1

In [55]:
'ENSG00000090263' not in true_classes.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].index

True

In [56]:
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000112234'), 'true_class'] == 1.0
assert predixcan_classifier_df.loc[('M41-Diagnoses_main_ICD10_M41_Scoliosis', 'ENSG00000090263'), 'true_class'] == 0.0

In [57]:
len(gwas2gene_all_genes)

9722

In [59]:
# score
df_score = pd.Series(index=classifier_index)

for trait in ukb_traits_common:
    trait_code = Trait(full_code=trait).code
    if trait_code not in gwas2genes_results:
        print(trait_code)
        continue
    trait_genes = gwas2genes_results[trait_code]
    scores = spredixcan_genes_associations.loc[trait_genes, trait]
    df_score.loc[trait] = scores.values

M13
I95
I71
20002_1264
20002_1081
22130
C80
20002_1538


In [60]:
# some testing
df_score = df_score.dropna().sort_index()
assert df_score.isna().sum().sum() == 0

In [61]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    2.563019e-02
                             ENSG00000003756    7.435000e-02
                             ENSG00000004534    4.208160e-01
                             ENSG00000004838    4.417000e-04
                             ENSG00000004897    1.116773e-09
dtype: float64

In [62]:
# some testing
_gene, _trait = ('ENSG00000090263', 'M41-Diagnoses_main_ICD10_M41_Scoliosis')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [63]:
_gene, _trait = ('ENSG00000070061', 'O14-Diagnoses_main_ICD10_O14_Gestational_pregnancyinduced_hypertension_with_significant_proteinuria')
assert spredixcan_genes_associations.loc[_gene, _trait] == df_score.loc[_trait, _gene]

In [64]:
df_score.shape

(19689,)

In [65]:
df_score.head()

ukb_efo                      gene           
1200-Sleeplessness_insomnia  ENSG00000001617    2.563019e-02
                             ENSG00000003756    7.435000e-02
                             ENSG00000004534    4.208160e-01
                             ENSG00000004838    4.417000e-04
                             ENSG00000004897    1.116773e-09
dtype: float64

In [66]:
df_score.min()

0.0

In [67]:
df_score.max()

1.759235608

In [68]:
predixcan_classifier_df = predixcan_classifier_df.assign(score=df_score)

In [70]:
# assert not predixcan_classifier_df['score'].isna().any()

In [71]:
from scipy import stats

In [72]:
_n_genes = len(gwas2gene_all_genes)
display(_n_genes)

SCORE_THRESHOLD = 0.1
display(SCORE_THRESHOLD)

9722

0.1

In [73]:
predixcan_classifier_df = predixcan_classifier_df.assign(predicted_class=(predixcan_classifier_df['score'] > SCORE_THRESHOLD).astype(int))

In [74]:
predixcan_classifier_df.shape

(19709, 3)

In [75]:
predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [76]:
predixcan_classifier_df.loc['M41-Diagnoses_main_ICD10_M41_Scoliosis'].sort_values('true_class', ascending=False).head()

,score,predicted_class,true_class
gene,,,
ENSG00000157764,6.912200e-08,0,1
ENSG00000162769,3.231060e-06,0,1
ENSG00000112234,6.222500e-07,0,1
ENSG00000012504,1.772800e-11,0,0
ENSG00000173208,1.777500e-07,0,0


## Select genes per trait

In [77]:
#selected_predixcan_classifier_df = predixcan_classifier_df.loc[predixcan_classifier_df.index.intersection(trait_genes_to_keep)]
selected_predixcan_classifier_df = predixcan_classifier_df

In [78]:
# some testing

In [79]:
selected_predixcan_classifier_df.shape

(19709, 3)

In [80]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [81]:
selected_predixcan_classifier_df.sort_values('predicted_class', ascending=False).head()

score  \
ukb_efo                                            gene                        
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000146830  0.895277   
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000148120  0.156800   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000008283  0.307305   
                                                   ENSG00000008083  0.203300   
                                                   ENSG00000006047  0.187403   

                                                                    predicted_class  \
ukb_efo                                            gene                               
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000146830                1   
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000148120                1   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000008283                1   
                                                   ENSG00000008083                1   
                                                   ENSG00000006047                1   

                                                                    true_class  
ukb_efo                                            gene                         
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000146830           0  
I48-Diagnoses_main_ICD10_I48_Atrial_fibrillatio... ENSG00000148120           0  
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000008283           0  
                                                   ENSG00000008083           0  
                                                   ENSG00000006047           0

In [82]:
_tmp = selected_predixcan_classifier_df.sort_values(['true_class', 'ukb_efo'], ascending=False)
display(_tmp.shape)
display(_tmp[_tmp['true_class'] > 0].shape)
display(_tmp[_tmp['true_class'] > 0].head())

(19709, 3)

(126, 3)

score  \
ukb_efo                                            gene                            
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305  5.758000e-06   
                                                   ENSG00000140443  4.758100e-06   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  5.454000e-11   
                                                   ENSG00000183287  1.143380e-10   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183  2.062000e-03   

                                                                    predicted_class  \
ukb_efo                                            gene                               
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305                0   
                                                   ENSG00000140443                0   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812                0   
                                                   ENSG00000183287                0   
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183                0   

                                                                    true_class  
ukb_efo                                            gene                         
R69-Diagnoses_main_ICD10_R69_Unknown_and_unspec... ENSG00000068305           1  
                                                   ENSG00000140443           1  
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812           1  
                                                   ENSG00000183287           1  
N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney... ENSG00000131183           1

### Test classes

In [83]:
selected_predixcan_classifier_df.index.get_level_values('ukb_efo').unique().shape

(99,)

In [84]:
selected_predixcan_classifier_df.index.get_level_values('gene').unique().shape

(9722,)

In [85]:
_pheno = 'N20-Diagnoses_main_ICD10_N20_Calculus_of_kidney_and_ureter'

In [86]:
_clinvar_asthma_genes = omim_silver_standard[omim_silver_standard['trait'] == _pheno]['ensembl_gene_id'].unique()
display(_clinvar_asthma_genes)
display(_clinvar_asthma_genes.shape)

array(['ENSG00000075891', 'ENSG00000130600', 'ENSG00000269821',
       'ENSG00000129757', 'ENSG00000143473', 'ENSG00000275410',
       'ENSG00000019186', 'ENSG00000134371', 'ENSG00000036828',
       'ENSG00000088256', 'ENSG00000187091', 'ENSG00000122194',
       'ENSG00000138592', 'ENSG00000040531', 'ENSG00000138079',
       'ENSG00000021488', 'ENSG00000090402', 'ENSG00000131482',
       'ENSG00000137700', 'ENSG00000164007', 'ENSG00000113946',
       'ENSG00000116039', 'ENSG00000168000', 'ENSG00000123191',
       'ENSG00000171365', 'ENSG00000165704', 'ENSG00000042753',
       'ENSG00000075643', 'ENSG00000169692', 'ENSG00000109667',
       'ENSG00000131183', 'ENSG00000109062', 'ENSG00000149257',
       'ENSG00000198931', 'ENSG00000157388', 'ENSG00000167207',
       'ENSG00000124827', 'ENSG00000134873'], dtype=object)

(38,)

In [87]:
_tmp = selected_predixcan_classifier_df.loc[_pheno]
_tmp.loc[_tmp.index.intersection(_clinvar_asthma_genes)]

,score,predicted_class,true_class
ENSG00000131183,0.002062,0,1


In [88]:
_predixcan_asthma_genes = selected_predixcan_classifier_df.loc[_pheno]

In [89]:
_predixcan_asthma_genes.head()

,score,predicted_class,true_class
gene,,,
ENSG00000005187,2.666423e-09,0,0
ENSG00000005189,3.437330e-09,0,0
ENSG00000011638,3.278863e-09,0,0
ENSG00000027847,2.658019e-08,0,0
ENSG00000048140,1.244200e-08,0,0


In [90]:
selected_predixcan_classifier_df.shape

(19709, 3)

In [91]:
selected_predixcan_classifier_df['predicted_class'].value_counts()

0    18679
1     1030
Name: predicted_class, dtype: int64

In [92]:
selected_predixcan_classifier_df['true_class'].value_counts()

0    19583
1      126
Name: true_class, dtype: int64

In [93]:
selected_predixcan_classifier_df.sort_values(['true_class'], ascending=[False])

score  \
ukb_efo                                            gene                            
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000179218  7.260000e-04   
                                                   ENSG00000111252  7.058000e-02   
E66-Diagnoses_main_ICD10_E66_Obesity               ENSG00000176842  1.501600e-10   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000111057  7.881000e-10   
22127-Doctor_diagnosed_asthma                      ENSG00000232810  1.597013e-07   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000124299  4.289000e-06   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000125730  5.730000e-04   
20002_1456-Noncancer_illness_code_selfreported_... ENSG00000196735  3.151000e-10   
R40-Diagnoses_main_ICD10_R40_Somnolence_stupor_... ENSG00000133812  5.454000e-11   
M05-Diagnoses_main_ICD10_M05_Seropositive_rheum... ENSG00000204498  9.828148e-04   
E66-Diagnoses_main_ICD10_E66_Obesity               ENSG00000140718  1.942400e-09   
20002_1381-Noncancer_illness_code_selfreported_... ENSG00000244731  6.175005e-01   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000144061  3.351189e-04   
20002_1111-Noncancer_illness_code_selfreported_... ENSG00000143631  8.395043e-19   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000141646  9.968244e-07   
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_... ENSG00000185950  5.952480e-07   
M06-Diagnoses_main_ICD10_M06_Other_rheumatoid_a... ENSG00000204498  1.959000e-04   
1980-Worrier_anxious_feelings                      ENSG00000179344  2.197727e-05   
20002_1075-Noncancer_illness_code_selfreported_... ENSG00000111252  7.775000e-02   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000008441  7.031763e-32   
20002_1226-Noncancer_illness_code_selfreported_... ENSG00000166037  8.715000e-05   
20002_1065-Noncancer_illness_code_selfreported_... ENSG00000231852  3.831202e-05   
2453-Cancer_diagnosed_by_doctor                    ENSG00000064012  8.136001e-01   
20002_1456-Noncancer_illness_code_selfreported_... ENSG00000179344  5.187000e-02   
20002_1381-Noncancer_illness_code_selfreported_... ENSG00000166278  2.921000e-01   
E04-Diagnoses_main_ICD10_E04_Other_nontoxic_goitre ENSG00000178919  2.489005e-02   
2453-Cancer_diagnosed_by_doctor                    ENSG00000003400  3.455060e-06   
20002_1535-Noncancer_illness_code_selfreported_... ENSG00000109099  1.575788e-11   
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_... ENSG00000204217  1.864232e-07   
                                                   ENSG00000130164  5.682000e-01   
...                                                                          ...   
20002_1075-Noncancer_illness_code_selfreported_... ENSG00000108798  3.384000e-04   
                                                   ENSG00000108511  2.172140e-04   
                                                   ENSG00000108468  4.881407e-03   
                                                   ENSG00000108465  4.686941e-03   
                                                   ENSG00000108439  4.826876e-05   
                                                   ENSG00000104892  5.872399e-05   
                                                   ENSG00000104884  9.682000e-06   
                                                   ENSG00000104881  5.675020e-06   
                                                   ENSG00000104879  5.898500e-06   
                                                   ENSG00000104866  1.367676e-05   
                                                   ENSG00000111245  1.135016e-04   
                                                   ENSG00000111271  2.106055e-03   
                                                   ENSG00000120093  3.393410e-04   
                                                   ENSG00000111275  1.117659e-03   
                                                   ENSG00000120075  3.463566e-04   
                                           

# Save classifier table

In [94]:
# remove nans
selected_predixcan_classifier_df = selected_predixcan_classifier_df.dropna()

In [95]:
selected_predixcan_classifier_df.head()

score  predicted_class  \
ukb_efo                     gene                                             
1200-Sleeplessness_insomnia ENSG00000001617  2.563019e-02                0   
                            ENSG00000003756  7.435000e-02                0   
                            ENSG00000004534  4.208160e-01                1   
                            ENSG00000004838  4.417000e-04                0   
                            ENSG00000004897  1.116773e-09                0   

                                             true_class  
ukb_efo                     gene                         
1200-Sleeplessness_insomnia ENSG00000001617           0  
                            ENSG00000003756           0  
                            ENSG00000004534           0  
                            ENSG00000004838           0  
                            ENSG00000004897           0

In [96]:
selected_predixcan_classifier_df.shape

(19689, 3)

In [97]:
selected_predixcan_classifier_df.to_csv(
    os.path.join(output_dir, 'fastenloc-torus-classifier_data.tsv.gz'),
    sep='\t', index=False
)